## FINAL VOLUMES

In [1]:
import random
import scipy.stats as st
import numpy as np
import math
from math import gamma, pi
import time
import scipy
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy.random as rnd
import pickle
import os.path
from deap import creator, base, tools, algorithms
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as la

In [2]:
def getAllSpansEqual(numOfDims, spanForAll):
    return np.ones(numOfDims)*spanForAll

def getRandomUniformSpans(numOfDims, low, high):
    return np.random.rand(numOfDims)*(high-low) + low

def getVolumeElipsoid(params):
    nDims = params.size
    return pow(pi, (nDims/2)) / gamma(nDims/2 + 1) * np.prod(params)

def getVolumeElipsoid2(params):
    nDims = params.size
    return (2/nDims)*pow(pi, (nDims/2)) / gamma(nDims/2) * np.prod(params)

# print(getVolumeElipsoid(np.array( [1,5,4])))
# print(getVolumeElipsoid2(np.array( [1,5,4])))

def fitEllipsoid_dumb(points, elipsoidParameters):
    d = np.size(elipsoidParameters)
    elipsoidParameters.shape=(1,d)
    
    max_dist = np.max( np.sum(np.square(np.divide(points,elipsoidParameters)),1) )
    increase_factor = math.sqrt(max_dist) + 0.00001
    print("increase_factor", increase_factor)
    new_ellipsoid_parameters = increase_factor*elipsoidParameters
    return new_ellipsoid_parameters

def isInElipsoid(points, elipsoidParameters):
    # points is a d x num_p numpy matrix where d is the number of dimensions and num_p is the number of points.
    # elipsoidParameters is an d-dimensional array, where each element is the RADIUS of the axis.
    d = np.size(elipsoidParameters)
    elipsoidParameters.shape=(1,d) 
    return np.sum(np.square(np.divide(points,elipsoidParameters)),1) <= 1

In [3]:
def minVolEllipse(P, tolerance ):
    # P = d x numP ( points )
    # tolerance = skalar ( priporočeno = 0.0004 )

    d, N = np.shape(P)
    Q = np.ones((d+1, N))
    Q[0:d,:] = P

    # initializations
    # -----------------------------------
    count = 1
    err = 1
    u = (1/N) * np.ones((N,1))          # 1st iteration
    
    while (err > tolerance):
        X = np.dot( np.dot( Q   , np.diagflat(u))    , Q.T ) # % X = \sum_i ( u_i * q_i * q_i')  is a (d+1)x(d+1) matrix
        M = (np.dot( Q.T , np.linalg.inv(X) ) * Q.T).sum(-1)
        # print("M", M)
        M.shape=(N,1)
        j = np.argmax(M)
        maximum = M[j,0]
        step_size = (maximum - d -1)/((d+1)*(maximum-1))
        new_u = (1 - step_size)*u
        new_u[j] = new_u[j] + step_size
        count += 1
        err = np.linalg.norm(new_u - u)
        u = new_u
        print("err", err)
        
    U = np.diagflat(u)
    print("done, err final =", err, ", iterations needed:", count )

    # the A matrix for the ellipse
    # --------------------------------------------
    Pu = np.dot(P,u)
    C = (1/d) * np.linalg.pinv( np.dot( np.dot( P , U) , P.T ) - np.dot(Pu, Pu.T ))

    # center of the ellipse 
    # --------------------------------------------
    b = np.dot(P , u)
    return C, b

In [4]:
def getAllVolumesAll10PCA( filesArray, bits0123, PCA_num):
    # GET POINTS OF THE FIRST PCA_num ITERATIONS OF THE PCA.
    model_index = bits0123     # 4 bit processor if bits0123=3!
    model_str = '0'+str(model_index+1)+'_'
        
    ga_solutions = False
    local_solutions = True
    region_files = []
    
    for file_name_str in filesArray:
        base_path_opt = os.path.join("..", file_name_str )
        
        if ga_solutions:
            region_files.append(os.path.join(base_path_opt, model_str+"bioprocViableSet_IterGA.p"))
        if local_solutions:
            for i in range(0, PCA_num):
                region_files.append(os.path.join(base_path_opt, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))

    print(region_files)

    viablePoints = []  
    for region_file in region_files: 
        viablePointsRegion = pickle.load(open(region_file, "rb")) 
        viablePoints.extend(viablePointsRegion)

    viablePoints = np.array(viablePoints)

    print("Number of points ("+str(model_index+1)+"-bit):",len(viablePoints))
    print("Shape of viable points:", viablePoints.shape)

    nDims = viablePoints.shape[1]

    pca = PCA(n_components=nDims)
    pca.fit( viablePoints )
    transformedViable = pca.transform(viablePoints)

    minP = np.min(transformedViable, axis=0)
    maxP = np.max(transformedViable, axis=0)
    dP = maxP - minP
    volCube = np.prod(dP) 

    radiuses_0 = dP/2
    center = (maxP + minP)/2

    radiuses_dumb = fitEllipsoid_dumb(transformedViable, radiuses_0)
    # print(radiuses_dumb)
    score_dumb = np.max(np.sum(np.square(np.divide( transformedViable, radiuses_dumb )),1))
    vol_dumb = getVolumeElipsoid(radiuses_dumb)
    vol_dumb_frac = vol_dumb / volCube

    ########################################################################################
    
    P = transformedViable.T
    tolerance = 0.001

    C, b = minVolEllipse(P, tolerance )

    C_minus = C/(1-np.dot( b.T, np.dot(C,b) )  )
    (eigvals,eigvecs) = la.eig(C_minus)
    P2 = np.dot( eigvecs.T, (P - b))

    R = 1./np.sqrt(np.abs(eigvals))
    R.shape = (np.size(R), 1)

    R2 = R*np.sqrt( np.max(np.sum(np.square(np.divide( P2, R )),0)) )
    score_good = np.max(np.sum(np.square(np.divide( P2, R2 )),0))
    
    vol_good = getVolumeElipsoid(R2)
    vol_good_frac = vol_good / volCube
    vol_good_vs_dumb = vol_good / vol_dumb
    
    print("score_good   :", score_good)
    print("score_dumb   :",  score_dumb)
    print("volCube      :", volCube)
    print("Vol_dumb     :", vol_dumb)
    print("dumb_/_cube  :", vol_dumb_frac)
    print("vol_good     :", vol_good)
    print("good_/_cube  :", vol_good_frac )
    print("good_/_dumb  :", vol_good_vs_dumb )

    volumes = [volCube, vol_dumb, vol_dumb_frac, vol_good, vol_good_frac, vol_good_vs_dumb ]
    
    return volumes


In [10]:
filesArray = [ "results_opt", "results_opt_rep1", "results_opt_rep2" ]
bits0123 = 0
PCA_num = 10

getAllVolumesAll10PCA( filesArray=filesArray , bits0123=bits0123, PCA_num=PCA_num)

['..\\results_opt\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0033289899990953916
err 0.0033025380129780043
err 0.003159107320681019
err 0.0031997371911412454
err 0.0034104604042711584
err 0.0032738571575736397
err 0.0033568062880580613
err 0.0035136623591137195
err 0.0034642675019403704
err 0.0035978128075017414
err 0.003738865928822512
err 0.003541008540949292
err 0.003149231198519234
err 0.0031499625617453617
err 0.0029556203066104974
err 0.0031853971416710134
err 0.003006263454874774
err 0.0032521043683922733
err 0.002871739804481317
err 0.003124662692588347
err 0.003128625238780015
err 0.0033642881176436244
err 0.003397581992680487
err 0.0033852842313499333
err 0.0034525731616880315
err 0.0035937210854612846
err 0.0035715927193360236
err 0.0037507382896314763
err 0.00369895365092339
err 0.0033438899329178247
err 0.0035337831931480826
err 0.0031935240709935492
err 0.0033339358036105575
err 0.0033210516600660866
err 0.0031626536401201514
err 0.0031900785101177348
err 0.0034108278282279332
err 0.0028893886369546636
err 0.00284827303327880

err 0.0015731367508833606
err 0.001594946461136005
err 0.0015862480779343154
err 0.0017265467872846156
err 0.001634587603449287
err 0.0017204914472920543
err 0.001515394498782616
err 0.0015225039162405061
err 0.0015333850168348707
err 0.0016595118128450537
err 0.0016993344348419591
err 0.001782568935663929
err 0.0018083747644620837
err 0.0015153950131993061
err 0.0015835472926295248
err 0.0016395103861993289
err 0.001517244078239645
err 0.0016324907311570597
err 0.0016442517251193545
err 0.0016698615528212984
err 0.0015124052738717416
err 0.001573483530322129
err 0.001517505810251036
err 0.001468734740050672
err 0.0015568484938713822
err 0.0014619526315592809
err 0.001483486209688284
err 0.00140916301579652
err 0.0014582042056028425
err 0.0015732902104555835
err 0.0016211420197321334
err 0.0014875715165871637
err 0.00157421006214876
err 0.0014960633193955377
err 0.001600765459678574
err 0.0017260003485351852
err 0.0016845061719959137
err 0.0017902352233793496
err 0.0015846965973256484


[1146920970994555.8,
 7906305265206544.0,
 6.893504840486584,
 115542126933739.17,
 0.10074114072005022,
 0.014613921807725793]

In [11]:
filesArray = [ "results_opt", "results_opt_rep1", "results_opt_rep2" ]
bits0123 = 1
PCA_num = 10

getAllVolumesAll10PCA( filesArray=filesArray , bits0123=bits0123, PCA_num=PCA_num)

['..\\results_opt\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0036627241857147734
err 0.003671712632656655
err 0.003952050326577935
err 0.003892013606402139
err 0.00375019759579462
err 0.003650617047002969
err 0.0039047629375529463
err 0.0037596614964295295
err 0.0036874306306573933
err 0.0037292741290779513
err 0.003972819076214352
err 0.003125934271736365
err 0.003445912575988504
err 0.003363499638830676
err 0.0030474886481260516
err 0.0030802746004894715
err 0.0032407054547443485
err 0.003137992901859739
err 0.0032997868219892075
err 0.003274233192559189
err 0.003304046378284787
err 0.0031352337421338204
err 0.0032857817853802337
err 0.0034005659369768814
err 0.0033599506522707435
err 0.0031562639198085417
err 0.003285058301546885
err 0.0032855335150757995
err 0.00333320057156689
err 0.0030538075500738345
err 0.00277973220795063
err 0.0029184845551470276
err 0.0027267292847824695
err 0.002723856202124775
err 0.002928453818017156
err 0.0031242377049781863
err 0.0029350653430410462
err 0.0029885240641488843
err 0.0029642903384660474
err 0.

err 0.001724475874334121
err 0.001739265134450841
err 0.0015808449561573528
err 0.0015331775686385383
err 0.001657504123080956
err 0.0015259760383290433
err 0.0016153860637327729
err 0.0016793366171753215
err 0.0014979302635783056
err 0.0015136217667874
err 0.0015978288868862365
err 0.001612203634380026
err 0.0016212495067513008
err 0.0016293664755570709
err 0.0016704789575223155
err 0.0017414251567151415
err 0.0016686731353180622
err 0.0015787268001319942
err 0.0015698270344159106
err 0.0016154110765642083
err 0.0016885077327076332
err 0.001636471735360298
err 0.0015703988242652997
err 0.0016663955313420283
err 0.0015992970767925638
err 0.001634808281285814
err 0.0016075742712814433
err 0.0015116018555612737
err 0.0015906109178623214
err 0.0015544098347186285
err 0.0015021737145921382
err 0.0015687009455586155
err 0.0014502963906361944
err 0.0014836722801089698
err 0.0015524305111061347
err 0.0016244410317749903
err 0.0014841637851760528
err 0.0014863839554359777
err 0.001519814019357

[144849568162577.28,
 196055770863929.44,
 1.35351298143933,
 10865278251976.707,
 0.075010774210812,
 0.05541932381841311]

In [12]:
filesArray = [ "results_opt", "results_opt_rep1", "results_opt_rep2" ]
bits0123 = 2
PCA_num = 10

getAllVolumesAll10PCA( filesArray=filesArray , bits0123=bits0123, PCA_num=PCA_num)

['..\\results_opt\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0037502735874668826
err 0.0037673237105557856
err 0.0036364875042373063
err 0.0035985367328335263
err 0.003646968482112623
err 0.003586466665340486
err 0.003448913240247261
err 0.0035087144728934054
err 0.003504901801122898
err 0.0037151549063240536
err 0.003460098244525504
err 0.0035415837717561426
err 0.003444136923425677
err 0.003578700685660873
err 0.003492074176234523
err 0.003387401542981015
err 0.0031292332546848287
err 0.003302444387564298
err 0.0032492274682926077
err 0.003331366074066163
err 0.003352829631455612
err 0.003292518483542293
err 0.003438520851719692
err 0.0033256979579966155
err 0.003485080251820534
err 0.003127506200368126
err 0.003147353834396307
err 0.002991029841225186
err 0.002986111522453871
err 0.003116165671362214
err 0.0030278355892260885
err 0.002848578009426261
err 0.0027773327041469187
err 0.002937394849807113
err 0.0028368401111784894
err 0.002992444238997511
err 0.0030916500305152523
err 0.003245746784348456
err 0.003067011698560861
err 0.00308

err 0.001606412904329894
err 0.001435145196745755
err 0.00157010815139386
err 0.0015851749525877172
err 0.0016507535730421446
err 0.001630633978130193
err 0.0016768131806314526
err 0.001594864280470532
err 0.0016282456980838682
err 0.0016853709777257823
err 0.001732977918116515
err 0.0015476835722192876
err 0.001731259495476562
err 0.0014958745634281155
err 0.0015375105684346567
err 0.0014020789541129251
err 0.0015115866849065031
err 0.001483946606974244
err 0.0014302532882601259
err 0.001486771039021849
err 0.0015657001035435166
err 0.001552075606624649
err 0.0015679576601246715
err 0.0015729477336417137
err 0.001605926224840916
err 0.0016618829976489365
err 0.0014549327703130973
err 0.0015468852600742356
err 0.0015802865342971271
err 0.0014503107803255134
err 0.0014745619795047023
err 0.001516754757924641
err 0.0015624354902929146
err 0.0015069335657170889
err 0.0014497202992756586
err 0.0014646150253744355
err 0.001575311913889132
err 0.0016431628271110473
err 0.0015376454463393674


[201793447387306.44,
 848337534568564.4,
 4.203989502891698,
 17082689112801.904,
 0.0846543301280479,
 0.020136665438821564]

In [13]:
filesArray = [ "results_opt", "results_opt_rep1", "results_opt_rep2" ]
bits0123 = 3
PCA_num = 10

getAllVolumesAll10PCA( filesArray=filesArray , bits0123=bits0123, PCA_num=PCA_num)

['..\\results_opt\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0035148636765535844
err 0.003340552262643593
err 0.0033026687341656696
err 0.003309171226917778
err 0.0033721542010498582
err 0.0034626998356674797
err 0.003404633210752619
err 0.00315822435731037
err 0.0030671807716434805
err 0.0033275618717295576
err 0.0033876786002611657
err 0.003531945941446171
err 0.0037793417422366743
err 0.003768796757999577
err 0.003590023649528355
err 0.0036970211577954507
err 0.0037538100131200786
err 0.003697554699387193
err 0.00377787106972009
err 0.0035082613750220576
err 0.003159194309807006
err 0.0032356757614088448
err 0.0033037003132577454
err 0.0029979654694858486
err 0.0031597103779078344
err 0.0031753563934235006
err 0.0033096415608543216
err 0.003103321691140562
err 0.0032074433306536827
err 0.0030113415633781448
err 0.003118313798715249
err 0.0033016191922679516
err 0.003438066200480117
err 0.003502288796187878
err 0.0028560241080679625
err 0.0031544274109326057
err 0.0032664859564794447
err 0.003101298506454657
err 0.00324942799237503
err 0

err 0.0014304668461958359
err 0.001434006134769545
err 0.0013948028076995534
err 0.0014723491503494254
err 0.0015640588335033073
err 0.00154755668150083
err 0.0015501659259850886
err 0.0016231739511209015
err 0.0016225328621438046
err 0.0016289912654897203
err 0.0015632280293440908
err 0.0015901499120553168
err 0.0015268215650727167
err 0.0015488032189627187
err 0.001637930426549989
err 0.001628702217863119
err 0.0015758205978387754
err 0.0017035897186855361
err 0.0017386695005276282
err 0.0013627137088356807
err 0.0014791935799136023
err 0.0015743564992225071
err 0.0016699130973784819
err 0.001676457815214518
err 0.0015804152812298178
err 0.0016396857314541601
err 0.0016949581248692249
err 0.0016848199682468895
err 0.0015208814973393103
err 0.0015512748696708688
err 0.0014436698888824423
err 0.0014053991339424707
err 0.0014752722300295366
err 0.0015610760729314029
err 0.0016820442043077506
err 0.0015803954602485323
err 0.001650116878664429
err 0.0015770237569404325
err 0.0015830550161

[290902315023098.3,
 1450295131850007.0,
 4.9855056386706655,
 13586210714596.467,
 0.0467036871587553,
 0.009367893759159075]